In [ ]:
import numpy as np
import time
import os
import matplotlib.pyplot as plt

from multirtd.RTD_planner import RTD_Planner
from multirtd.zonotope import Zonotope
import multirtd.params as params

%load_ext autoreload
%autoreload 2

In [ ]:
# Initialize planner
p_0 = np.array([[-5], [0]])
LPM_file = os.path.join(os.getcwd(),'..', 'models', 'quadrotor_linear_planning_model.mat')
planner = RTD_Planner(LPM_file, p_0)

In [ ]:
# Set planner goal and obstacles
planner.p_goal = np.array([[5], [0]])
# (center, radius) tuples
planner.obstacles = [Zonotope(np.array([2.5, 1.5])[:,None], 0.5*np.array([[1, 1], [1, -1]])),
                     Zonotope(np.array([0, -1.5])[:,None], 0.5*np.array([[1, 1], [1, -1]])),
                     Zonotope(np.array([-2.5, 1.5])[:,None], 0.5*np.array([[1, 1], [1, -1]]))]

In [ ]:
# Visualize setup
fig, ax = plt.subplots(figsize=(12,6))
ax.set_xlim((-6, 6))
ax.set_ylim((-3, 3))
ax.grid()

#ax.scatter(p_0[0], p_0[1], s=50, marker='^', color='b')
ax.add_patch(plt.Circle(tuple(p_0), params.R_BOT, color='b'))
ax.scatter(planner.p_goal[0], planner.p_goal[1], s=100, marker='*', color='g')

for obs in planner.obstacles:
    obs.plot(ax, color='r')

plt.show()

In [ ]:
# "Controller" loop frequency 
# Determined by trajectory discretization
# Currently the controller just pops off setpoints from the planner trajectories to save for plotting
controller_hz = 10  # 0.1 s

# Planner loop frequency
# Determines time between replans
planner_hz = 2  # 0.5 s

t_sim = 0  # counter for simulation time
trajectory = None
traj_idx = 0
done = False

# Initialize state
p = p_0
v = np.zeros((2,1))
a = np.zeros((2,1))
# State history
P = []
planning_times = []
FRS = []

while not done:

    # Planner
    if t_sim % int(controller_hz/planner_hz) == 0:
        # Replan
        print("Replanning: t_sim = ", t_sim)
        start_t = time.time()
        trajectory = planner.replan((p,v,a))
        planning_times.append(time.time() - start_t)
        traj_idx = 0

    # Controller
    # Pop off next setpoint
    p = trajectory[0][:,traj_idx][:,None]
    v = trajectory[1][:,traj_idx][:,None]
    a = trajectory[2][:,traj_idx][:,None]
    P.append(p)
    traj_idx += 1

    FRS.append(trajectory[3][traj_idx])

    if np.linalg.norm(p - planner.p_goal) < params.R_GOAL_REACHED:
        print("Goal reached")
        done = True

    t_sim += 1

P = np.hstack(P)
print("Average planning time: ", np.mean(planning_times), " seconds")

In [ ]:
# Visualize setup
fig, ax = plt.subplots(figsize=(12,6))
ax.set_xlim((-6, 6))
ax.set_ylim((-3, 3))
ax.grid()

ax.add_patch(plt.Circle(tuple(p_0), params.R_BOT, color='b'))
ax.scatter(planner.p_goal[0], planner.p_goal[1], s=100, marker='*', color='g')

for obs in planner.obstacles:
    obs.plot(ax, color='r')

# Plot trajectory
ax.scatter(P[0,:], P[1,:], s=10, marker='o', color='b')

for z in FRS:
    z.view([0,1]).plot(ax, alpha=0.01)

plt.show()